In [2]:
import findspark
findspark.init('/home/spark/spark-2.1.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
from pyspark.ml.regression import LinearRegression

In [8]:
lr = LinearRegression(featuresCol = 'features',labelCol = 'label',predictionCol = 'prediction')

In [9]:
lrmodel = lr.fit(training)

In [11]:
lrmodel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
lrmodel.intercept

0.14228558260358093

In [14]:
training_summary = lrmodel.summary

In [15]:
training_summary.r2

0.027839179518600154

In [16]:
training_summary.rootMeanSquaredError

10.16309157133015

In [17]:
all_data  = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [19]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                347|
|   mean|0.26301640891278144|
| stddev| 10.267618347386048|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                154|
|   mean|0.24308185731788487|
| stddev| 10.463951099282355|
|    min|-26.805483428483072|
|    max| 27.111027963108548|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

In [24]:
test_results = correct_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-27.274067615631203|
|-22.445301391688353|
|-20.662008836701588|
| -20.36065954098265|
|-20.382194751973564|
|-18.868577094155313|
|-17.987767610708467|
|-15.487913784146976|
| -17.22574803912085|
| -19.75935946105337|
|-12.886080827529295|
|-13.824034605867418|
|-18.579544250981204|
|-12.673917476273727|
|-15.820208161687802|
| -14.21754128288795|
|-16.405524806902132|
|-12.387591747499034|
| -7.621962103700809|
|-13.348703489552362|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

10.78403934124971

In [30]:
unlabeled_data = test_data.select('features')

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = correct_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| 0.46858418714813255|
|(10,[0,1,2,3,4,5,...|  1.0129136275225488|
|(10,[0,1,2,3,4,5,...|  0.7890177986331829|
|(10,[0,1,2,3,4,5,...|  0.5778967513681106|
|(10,[0,1,2,3,4,5,...|  0.9798587217590113|
|(10,[0,1,2,3,4,5,...|  0.5933635281506833|
|(10,[0,1,2,3,4,5,...|  1.2955605893973625|
|(10,[0,1,2,3,4,5,...| 0.11205606083467909|
|(10,[0,1,2,3,4,5,...|  1.8662031592881738|
|(10,[0,1,2,3,4,5,...|   4.424591981131029|
|(10,[0,1,2,3,4,5,...| -2.4248997618869947|
|(10,[0,1,2,3,4,5,...| -0.9387236470637094|
|(10,[0,1,2,3,4,5,...|   4.807102689278333|
|(10,[0,1,2,3,4,5,...|-0.47941813009180356|
|(10,[0,1,2,3,4,5,...|  2.7802800975831863|
|(10,[0,1,2,3,4,5,...|   1.658965494031762|
|(10,[0,1,2,3,4,5,...|  3.9140827293557185|
|(10,[0,1,2,3,4,5,...| -0.0800646335338259|
|(10,[0,1,2,3,4,5,...|  -4.788984299139352|
|(10,[0,1,2,3,4,5,...|  1.916901